In [9]:
import cv2
import dlib
import numpy as np
from imutils import face_utils
import os

In [4]:
# Load the facial landmark predictor model
predictor_path = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

In [6]:
# Function to calculate the mouth aspect ratio (MAR)
def mouth_aspect_ratio(mouth):
    A = np.linalg.norm(mouth[2] - mouth[10])  # Distance between points 51 and 59
    B = np.linalg.norm(mouth[4] - mouth[8])   # Distance between points 53 and 57
    C = np.linalg.norm(mouth[0] - mouth[6])   # Distance between points 49 and 55
    mar = (A + B) / (2.0 * C)
    return mar

# Constants for detecting mouth opening
MOUTH_OPEN_THRESHOLD = 0.6
MOUTH_CLOSED_THRESHOLD = 0.4

In [8]:
# Start video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)

    for rect in rects:
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # Extract mouth landmarks (49-68)
        mouth = shape[48:68]

        # Calculate the mouth aspect ratio (MAR)
        mar = mouth_aspect_ratio(mouth)

        # Determine if the mouth is open or closed
        if mar > MOUTH_OPEN_THRESHOLD:
            mouth_status = "Mouth Open"
        else:
            mouth_status = "Mouth Closed"

        # Draw the mouth region and display MAR
        cv2.putText(frame, f'MAR: {mar:.2f}', (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(frame, mouth_status, (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.drawContours(frame, [mouth], -1, (0, 255, 0), 1)

    # Display the frame
    cv2.imshow('Mouth Open/Closed Detection', frame)

    # Exit loop on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [22]:
import cv2
import dlib
import numpy as np
from imutils import face_utils
import os

# Load the facial landmark predictor model
predictor_path = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# Function to calculate mouth aspect ratio (MAR)
def mouth_aspect_ratio(mouth):
    A = np.linalg.norm(mouth[2] - mouth[10])  # Distance between points 51 and 59
    B = np.linalg.norm(mouth[4] - mouth[8])   # Distance between points 53 and 57
    C = np.linalg.norm(mouth[0] - mouth[6])   # Distance between points 49 and 55
    mar = (A + B) / (2.0 * C)
    return mar

# Constants for detecting mouth opening/closing
MOUTH_OPEN_THRESHOLD = 0.6
MOUTH_CLOSED_THRESHOLD = 0.4
FRAMES_PER_WORD = 22  # Number of frames per word
OUTPUT_DIR = './dataset/name'  # Directory to save classified videos

# Ensure output directory exists
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Start video capture
cap = cv2.VideoCapture(0)
recording = False
frames = []
word_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)

    for rect in rects:
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # Extract mouth landmarks (49-68)
        mouth = shape[48:68]

        # Calculate the mouth aspect ratio (MAR)
        mar = mouth_aspect_ratio(mouth)

        # Detect mouth opening
        if mar > MOUTH_OPEN_THRESHOLD:
            if not recording:
                print("Mouth opened, start recording")
                recording = True
                frames = []  # Reset the frame buffer

            frames.append(frame)

        # Detect mouth closing
        elif mar < MOUTH_CLOSED_THRESHOLD and recording:
            print("Mouth closed, stop recording")
            recording = False

            if len(frames) >= FRAMES_PER_WORD:
                # Save captured frames as a video file
                video_path = os.path.join(OUTPUT_DIR, f'word_{word_count}.avi')

                # Define the codec and create VideoWriter object
                height, width, layers = frames[0].shape
                video = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'XVID'), 30, (width, height))

                for f in frames:
                    video.write(f)

                video.release()
                print(f"Saved video: {video_path}")
                word_count += 1  # Increment word count for the next video

    # Display the frame
    cv2.imshow('Mouth Detection', frame)

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Mouth opened, start recording
Mouth closed, stop recording
Mouth opened, start recording
Mouth closed, stop recording
Saved video: ./dataset/name\word_0.avi
Mouth opened, start recording
Mouth closed, stop recording
Mouth opened, start recording
Mouth closed, stop recording


In [25]:
import cv2
import os

# Constants
FRAMES_PER_VIDEO = 30  # Adjust the number of frames you want per video
OUTPUT_DIR = './dataset/name'  # Directory to save the videos

# Ensure output directory exists
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Start video capture
cap = cv2.VideoCapture(0)
recording = False
frames = []
word_count = 0

print("Press 'r' to start/stop recording and 'q' to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Display the frame
    cv2.imshow('Video Recording', frame)

    # Listen for key presses
    key = cv2.waitKey(1) & 0xFF

    if key == ord('r'):  # Start/Stop recording
        if not recording:
            print("Recording started...")
            recording = True
            frames = []  # Reset the frame buffer
        else:
            print("Recording stopped...")
            recording = False

            # Save captured frames as a video file
            video_path = os.path.join(OUTPUT_DIR, f'word_{word_count}.avi')

            # Define the codec and create VideoWriter object
            height, width, layers = frames[0].shape
            video = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'XVID'), 30, (width, height))

            for f in frames:
                video.write(f)

            video.release()
            print(f"Saved video: {video_path}")
            word_count += 1  # Increment word count for the next video

    elif key == ord('q'):  # Exit the program
        print("Exiting...")
        break

    # Capture frames if recording is active
    if recording:
        frames.append(frame)

        # Optional: limit to a fixed number of frames per video
        if len(frames) >= FRAMES_PER_VIDEO:
            print("Reached max frames for one video, saving...")
            recording = False

            # Save the video as before
            video_path = os.path.join(OUTPUT_DIR, f'word_{word_count}.avi')
            height, width, layers = frames[0].shape
            video = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'XVID'), 30, (width, height))

            for f in frames:
                video.write(f)

            video.release()
            print(f"Saved video: {video_path}")
            word_count += 1

cap.release()
cv2.destroyAllWindows()


Press 'r' to start/stop recording and 'q' to quit
Recording started...
Reached max frames for one video, saving...
Saved video: ./dataset/name\word_0.avi
Exiting...


In [61]:
import cv2
import os
import time

# Constants for capturing video
FRAMES_PER_WORD = 50  # Number of frames per word
WORDS_TO_CAPTURE = 11  # Number of words to capture
OUTPUT_DIR = './dataset/name'  # Directory to save classified videos
INTERVAL_SECONDS = 3  # Interval between each word in seconds

# Ensure output directory exists
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Start video capture
cap = cv2.VideoCapture(0) 
frame_count = 0
word_count = 0
frames = []

while word_count < WORDS_TO_CAPTURE:
    # Wait before starting the next recording
    if frame_count == 0 and word_count > 0:
        print(f"Waiting {INTERVAL_SECONDS} seconds before recording next word...")
        time.sleep(INTERVAL_SECONDS)
    
    ret, frame = cap.read()
    if not ret:
        break

    frames.append(frame)
    frame_count += 1

    # Once we have 30 frames (one word's worth of frames)
    if frame_count == FRAMES_PER_WORD:
        print("Recording...")
        
        # Save the frames as a video file
        video_path = os.path.join(OUTPUT_DIR, f'word_{word_count}.avi')

        # Define the codec and create VideoWriter object
        height, width, layers = frames[0].shape
        video = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'XVID'), 30, (width, height))

        for f in frames:
            video.write(f)

        video.release()
        print(f"Saved video: {video_path}")

        # Reset frame count and clear frame buffer for the next word
        frame_count = 0
        frames = []
        word_count += 1  # Move to the next word

    # Display the frame
    cv2.imshow('Frame Capture', frame)

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Recording...
Saved video: ./dataset/my\word_0.avi
Waiting 3 seconds before recording next word...
Recording...
Saved video: ./dataset/my\word_1.avi
Waiting 3 seconds before recording next word...
Recording...
Saved video: ./dataset/my\word_2.avi
Waiting 3 seconds before recording next word...
Recording...
Saved video: ./dataset/my\word_3.avi
Waiting 3 seconds before recording next word...
Recording...
Saved video: ./dataset/my\word_4.avi
Waiting 3 seconds before recording next word...
Recording...
Saved video: ./dataset/my\word_5.avi
Waiting 3 seconds before recording next word...
Recording...
Saved video: ./dataset/my\word_6.avi
Waiting 3 seconds before recording next word...
Recording...
Saved video: ./dataset/my\word_7.avi
Waiting 3 seconds before recording next word...
Recording...
Saved video: ./dataset/my\word_8.avi
Waiting 3 seconds before recording next word...
Recording...
Saved video: ./dataset/my\word_9.avi
Waiting 3 seconds before recording next word...
Recording...
Saved v